In [1]:
import pandas as pd
import os,re
from nltk import *
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer


In [2]:
doc_names = os.listdir("files")
docs = list()

In [3]:
for doc_name in doc_names:
    #Tokenization | these tokens includes punctuation marks also
    # tokens = list()
    # tokens = word_tokenize(file.read())
    #print(tokens)

    # These tokens will not contain punctuation marks
    tokenizer = RegexpTokenizer(r'\w+')
    text_without_urls = re.sub(r"http\S+", "", open("files/"+doc_name,encoding="UTF-8").read())
    tokens = tokenizer.tokenize(text_without_urls)
    
    # Writing tokens to output file
    tokens_file = open("tokens/"+doc_name+"_tokens.txt","w+")
    tokens_file.write(",".join(tokens))
    tokens_file.flush()
    tokens_file.close()
    
    # Opening & reading tokens file
    tokens_file = tokens_file = open("tokens/"+doc_name+"_tokens.txt","r+")
    tokens = tokens_file.read().split(",")
    
    #Stop Word Removal
    stop_words = stopwords.words('english')
    filtered_words = list()
    for token in tokens:
        if token not in stop_words:
            filtered_words.append(token)        
    
    # Writing reduced output to output file
    reduced_file = open("reduced/"+doc_name+"_reduced.txt","w+")
    reduced_file.write(",".join(filtered_words))
    reduced_file.flush()
    reduced_file.close()
    
    # Opening & reading reduced file
    reduced_file = open("reduced/"+doc_name+"_reduced.txt","r+")
    filtered_words = reduced_file.read().split(",")
    
    #POS tagging after stop words removal
    tagged_tokens = pos_tag(filtered_words)
    
    '''# Stemming (PorterStemming)
    stemmed_base_words = list()
    stemmer = PorterStemmer()
    for word in filtered_words:
        stemmed_base_words.append(stemmer.stem(word))
    # print(base_words)
    '''

    #Lemmitizing using WordNetLemmatizer
    lemmatized_base_words = list()
    lemmatizer = WordNetLemmatizer()

    for word,tag in tagged_tokens:
        word_net_tag = tag[0].lower()
                
        word_net_tag = word_net_tag if word_net_tag in ['a', 'r', 'n', 'v'] else None
        if not word_net_tag:
            lemmatized_base_words.append(word)
        else:
            lemmatized_base_words.append(lemmatizer.lemmatize(word,word_net_tag))
        
    ##print(lemmatized_base_words)
    preprocessed_file = open("preprocessed/"+doc_name+"_preprocessed.txt","w+");
    preprocessed_file.write(",".join(lemmatized_base_words))
    preprocessed_file.flush()
    preprocessed_file.close()
    
    preprocessed_file = open("preprocessed/"+doc_name+"_preprocessed.txt","r+");
    docs.append(preprocessed_file.read())
    preprocessed_file.close()

In [4]:
vec = CountVectorizer()
x = vec.fit_transform(docs)

frame = pd.DataFrame(x.toarray(),columns=vec.get_feature_names(), index=doc_names)

frame.to_csv("output/my_data_frame.csv")
print("Output File Generated: my_data_frame.csv")

Output File Generated: my_data_frame.csv


In [5]:
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()
sia.polarity_scores(open("files/doc1.txt").read())

{'neg': 0.0, 'neu': 0.95, 'pos': 0.05, 'compound': 0.5994}